In [16]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model


In [14]:
import os
import cv2
import numpy as np
from tqdm import tqdm

IMG_SIZE = 224

# Combine both image directories into a list
IMG_DIRS = ['Images/HAM10000_images_part_1', 'Images/HAM10000_images_part_2']

# Create a dictionary for quick image lookup
image_paths = {}
import pandas as pd

# Load metadata
df = pd.read_csv('Images/HAM10000_metadata.csv')

# Create binary labels: 'malignant' for (melanoma, BCC, AKIEC), else 'benign'
df['label'] = df['dx'].apply(lambda x: 'malignant' if x in ['mel', 'bcc', 'akiec'] else 'benign')


for dir in IMG_DIRS:
    for img_name in os.listdir(dir):
        if img_name.endswith('.jpg'):
            image_id = img_name.split('.')[0]
            image_paths[image_id] = os.path.join(dir, img_name)

def load_images_from_metadata(df):
    images = []
    labels = []
    for i, row in tqdm(df.iterrows(), total=len(df)):
        img_id = row['image_id']
        if img_id in image_paths:
            img_path = image_paths[img_id]
            img = cv2.imread(img_path)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(row['label'])
    return np.array(images), np.array(labels)

# Example usage (assuming you've already added 'label' column in metadata)
X, y = load_images_from_metadata(df)
X = X / 255.0  # Normalize


100%|████████████████████████████████████████████████████████████████████████████| 10015/10015 [02:57<00:00, 56.48it/s]


In [15]:
# Convert labels to 0 (benign) and 1 (malignant)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, stratify=y_encoded)

print(f"Train shape: {X_train.shape}, Val shape: {X_val.shape}")


Train shape: (8012, 224, 224, 3), Val shape: (2003, 224, 224, 3)


In [17]:
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model

# Custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1 (Conv2D)                │ (None, 112, 112, 32)      │             864 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bn_Conv1 (BatchNormalization) │ (None, 112, 112, 32)      │             128 │ Conv1[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1_relu (ReLU)             │ (None, 112, 112, 32)      │               0 │ bn_Conv1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise       │ (None, 112, 112, 32)      │             288 │ Conv1_relu[0][0]           │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_BN    │ (None, 112, 112, 32)      │             128 │ expanded_conv_depthwise[0… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_relu  │ (None, 112, 112, 32)      │               0 │ expanded_conv_depthwise_B… │
│ (ReLU)                        │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project         │ (None, 112, 112, 16)      │             512 │ expanded_conv_depthwise_r… │
│ (Conv2D)                      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project_BN      │ (None, 112, 112, 16)      │              64 │ expanded_conv_project[0][… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand (Conv2D)       │ (None, 112, 112, 96)      │           1,536 │ expanded_conv_project_BN[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_BN             │ (None, 112, 112, 96)      │             384 │ block_1_expand[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_relu (ReLU)    │ (None, 112, 112, 96)      │               0 │ block_1_expand_BN[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_pad (ZeroPadding2D)   │ (None, 113, 113, 96)      │               0 │ block_1_expand_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_depthwise             │ (None, 56, 56, 96)        │             864 │ block_1_pad[0][0]          │
│ (DepthwiseConv2D)             │                           │               

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [18]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32)


Epoch 1/5
251/251 ━━━━━━━━━━━━━━━━━━━━ 359s 1s/step - accuracy: 0.7774 - loss: 0.4970 - val_accuracy: 0.8218 - val_loss: 0.3825
Epoch 2/5
251/251 ━━━━━━━━━━━━━━━━━━━━ 316s 1s/step - accuracy: 0.8165 - loss: 0.3855 - val_accuracy: 0.8258 - val_loss: 0.3734
Epoch 3/5
251/251 ━━━━━━━━━━━━━━━━━━━━ 308s 1s/step - accuracy: 0.8336 - loss: 0.3696 - val_accuracy: 0.8198 - val_loss: 0.3766
Epoch 4/5
251/251 ━━━━━━━━━━━━━━━━━━━━ 327s 1s/step - accuracy: 0.8302 - loss: 0.3719 - val_accuracy: 0.8333 - val_loss: 0.3577
Epoch 5/5
251/251 ━━━━━━━━━━━━━━━━━━━━ 321s 1s/step - accuracy: 0.8411 - loss: 0.3610 - val_accuracy: 0.8357 - val_loss: 0.3617


In [19]:
loss, acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {acc:.2f}")



63/63 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - accuracy: 0.8305 - loss: 0.3737
Validation Accuracy: 0.84


In [20]:
model.save('skin_cancer_model.h5')
